# Lesson7 ニューラルネットでゲームを攻略するAIを作る

## Section1 解説

### 1.1 強化学習とは

強化学習とは、エージェントが環境と試行錯誤しながらより良い行動の学習をしていく方法。教師あり学習のように明示的に教師が与えられず、報酬という間接的な目標からのフィードバックを元に学習していく。

#### 1.1.1 具体的な例

**スマホの電波がいい場所を探す**

私たち(エージェント)は、スマホの電波が良い場所というのは分からず、さぐりさぐり歩きながら電波が良くなりそうな方向に歩いていく。

つまり、教師のようなもの(電波maxなスポットはどこか)というのは分からない一方、電波の良し悪しはスマホの画面を見ることで幾分か測ることが出来る。

(参考 : J. Si et al. Handbook of Learning and Approximate Dynamic Programming. Wiley-IEEE Press. 2004 )

**お金持ちになる**

お金持ちになるという目標に到達するために、私たち(エージェント)はいい職につくためにいい学校に入ろうとしたり、年収を上げるために転職したりする。同時に、その過程でその時点での所得がどれくらいなのか測りながら、その都度行動を修正していく。

また、ある地点で所得が上がりそうでない行動をした(ex. 留年をした、失職した)場合でも、後々それがよい結果につながったのではないかと考えられることもある。

#### 1.1.2 強化学習で取り扱う問題の性質

上記の例は、強化学習で扱うべき問題の典型例で、以下のような重要な要素を含んでいる。(教師あり学習と強化学習で相違点)

- 明確な教師が存在しない。
- 結果(報酬)が遅れて反映される。
- 探索と活用のトレードオフがある。
- 観測するデータがとった行動に依存する。

### 1.2 強化学習の定式化

強化学習では次のような定式化を行う。

行動する主体(Agent)が現在の状態(state)に基づいて行動(Action)を選択し、それを環境(Environment)が受けとり、新しい状態が生成され、Agentはそれを元にまた行動を選択するというプロセス。

前述のスマホの電波の例でいえば、エージェントは私達人間、行動はどの向きにどれくらい動くか、環境はスマホの電波マップ、報酬はその時点での電波の良し悪しという感じになる。

また、以下では時刻$t+1$ における状態$S_{t+1}$ 及び報酬$R_{t+1}$ は、時刻$t$ における状態$S_t$ 及び行動$A_t$ のみに依存すると仮定して考える。これを、**マルコフ性**を仮定する、という。

Q学習、DQNなどは、このマルコフ性を仮定したもとでのアルゴリズムである。

一見、マルコフ性を満たしていない環境、例えばブロック崩しの静止画だけでは玉がどの向きに動いているかわからないような状況においても、複数フレームをstackすれば問題ない。

![rl_concept](https://github.com/reminayano/matsuo/blob/master/lesson7/figures/rl_concept.png?raw=true)

この一連の繰り返しを**エピソード**という。

お金持ちの例で言うなら、1人生が1エピソードとなる。

エピソード的でない(終わりがない)タスクもあるが、ここでは扱わない。

#### 1.2.1 強化学習における目的

エージェントとしては、最終的に良い結果を得るためにどのような行動を取ったらいいかを学習していく。
この「どのような行動を取るか」についてのエージェントの指針を強化学習では**方策(policy, $\pi$ )**といい、いかに良い方策を学習できるかがポイントとなる。

具体的には、豊作は状態を入力とし、行動を出力とする関数となる。

強化学習では方策の良さを累積の利得で表現する。これを**利得**という。
利得を式で表すと以下の様になる。

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2R_{t+3} + …= \Sigma_{k=0}^\infty\gamma^kR_{t+k+1}
$$

ここで、$\gamma$ は割引率とよばれ、将来の報酬に対してどれだけ割り引くかの度合いを表す。

この利得(累積報酬)の期待値を最大化するような方策を求めることが強化学習の目的となる。
この、最大化するような方策を**最適方策**といい、以下の式で表される。

$$
\pi^* = \arg\max x_\pi E_\pi [\Sigma_{k=0}^\infty \gamma^k R_{t+k+1}]
$$

(arg maxf(x)：f(x) を最大にする x の集合)

#### 1.3.1 価値関数

価値関数とは、ある方策 **$\pi$** のもとで、各状態にいたとき・各行動をとったときに将来どれくらい利得を得ることができるかを示すもので、それぞれ**状態価値関数**、**行動価値関数**とよばれる。

**状態価値関数**

エージェントが状態$s$ にいたとき、方策 **$\pi$** のもとで将来得られる利得の期待値。

$$
v_\pi(s)=E_\pi[ R_{t+1} + \gamma R_{t+2} + \gamma^2R_{t+3} + …|S_t=s]=E_\pi[\Sigma_{k=0}^\infty\gamma^kR_{t+k+1}|S_t=s]
$$

**行動価値関数**

エージェントがある状態$s$ において行動$a$ をとったとき、方策$\pi$ のもとで得られる利得の期待値。

$$
q_\pi(s, a)=E_\pi[ R_{t+1} + \gamma R_{t+2} + \gamma^2R_{t+3} + …|S_t=s, A_t=a]=E_\pi[\Sigma_{k=0}^\infty\gamma^kR_{t+k+1}|S_t=s, A_t=a]
$$

#### 1.3.2 価値関数の更新

ここで、行動価値関数が次のように表すことができることに注目する。(状態価値関数でも同様)

$$
q_\pi(s, a)=E_\pi[G_t|S_t=s, A_t=a]=E_\pi[R_{t+1}+\gamma q_\pi (S_{t+1})|S_t=s, A_t=a]
$$

つまり、状態$s$ における価値関数は、その次のステップで得た報酬とその次の状態$s'$ の価値関数に割引をかけた和 $R_{t+1}+\gamma q_\pi (S_{t+1})$ で表すことが出来る。

ここで、**後者の方が実際の報酬 $R_{t+1}$ を観測している分、真の価値関数に近い**と考え、現在の推定値$Q_\pi(s)$ と新しい推定値$R_{t+1}+\gamma q_\pi (s')$ を利用して以下の様に更新を行う。

$$
Q(S_t, A_t)←Q(S_t, A_t) + \alpha[R_{t+1}+\gamma Q(S_{t+1}, A_{t+1})-Q(S_t, A_t)]
$$

$\alpha$ は更新幅を決めるハイパーパラメータである。

行動価値関数の場合、この更新方法は**Sarsa**とよばれる。

ここで、[]の中身(新しい推定値-古い推定値)はTD誤差とよばれる。

このように、**推定値からその時点での推定値をもとに更新を行う**ために、**Temporal Difference(TD)学習** とよばれる。

これが、TD学習の重要なポイントで、更新のターゲットに推定値を利用するので、エピソードが終結するのを待たずに価値関数を更新することが出来る。

#### 1.3.3 方策の更新

前述のとおり、強化学習の最終目的は最適方策を求めること。

そのため、価値関数の更新をおこなったあと、その価値関数のもとで各状態・行動において利得を最大化することが出来るように方策を更新していく。

具体的には、各状態において最も期待利得が大きくなるような行動を取る(greedy)ように更新を行う。

以下に方策の更新のイメージを示す。価値関数の更新と方策の更新を繰り返していく。

![gpi.png](https://github.com/reminayano/matsuo/blob/master/lesson7/figures/gpi.png?raw=true)

### 1.4 Q学習

Q学習もTD学習の一種だが、少し違う。

Sarsaではその時点での方策の価値関数を求めるように更新式を設定していたが、Q学習では直接最適方策をもとめるように更新を行う。

つまり、Sarsaでは価値関数の更新→方策の更新→価値関数の更新→方策の更新→…と繰り返していたが、Q学習では最適方策を直接ターゲットとして求めるため価値関数の更新のみを考えればよい。

具体的な更新式は以下のようになる。新しいい推定値の計算をgreedyな方策のもとで行うことで、最適方策における価値関数に直接近づけていく。

$$
Q(S_t, A_t)←Q(S_t, A_t)+\alpha [R_{t+1}+\gamma \max_aQ(S_{t+1}, a)-Q(S_t, A_t)]
$$

そのため、Sarsaでは価値推定をする方策と探索に使う方策は同じだったが、Q学習では価値推定する方策と探索に使う方策は違ってもよいということになる。

Sarsaのように価値推定する方策と探索に使う方策が同じ必要があるアルゴリズムを**on-policy**、Q学習のように同じ必要がないアルゴリズムを**off-policy**という。

Q学習では探索と活用のトレードオフを実現するため。ある一定の確率 $\epsilon$ ではランダムに行動し、$1-\epsilon$ でその時の最もよい行動を選択する**ε-greedy方策**を探索に用いるのが一般的である。

#### 1.5 Deep Q学習

今回のメインテーマであるDeep Q-Network(DQN)は、このQ学習をDeep Learningでうまく行うというもの。

DQNでは、状態を入力とし、各行動の価値関数を出力とするネットワークを構築する。このネットワークが**Q-Network**とよばれる。以下にQ-Networkのイメージ図を示す。

![dqn](https://github.com/reminayano/matsuo/blob/master/lesson7/figures/dqn.png?raw=true)

Q学習で用いたTD誤差を二乗したものを教師あり学習における誤差関数と考え、通常の勾配法などで最適方策求めていく。

- 誤差関数
$$
L=E[(R_{t+1}+\gamma \max_{a'}Q(S_{t+1}, a')-Q(S_t, A_t))^2]
$$

ここで、教師あり学習のように学習を進めるための工夫として、次の2つを導入する。

#### 1.5.1 Expensive Replay

前述のとおり、強化学習ではエージェントがとった行動によって次の状態・報酬が変化してきてしまうため、学習につかうサンプルの分布が安定しない。

DQNでは、エージェントは行動を進めつつ、実際の学習には過去それまでのデータからランダムにサンプリングしたものをパラメータの更新に使用する。こうすることで、サンプリングされるデータの分布を安定させる。

過去の経験から思い出すという意味で、これをExperience Replayという。

#### 1.5.2 Target Network

通常、教師あり学習は、教師信号が固定された状況でいかにそれにネットワークの出力を近づけるかを目標にしてパラメータの更新をしていた。しかし、前述のとおり、通常のQ学習では**推定値から推定を行う**という、教師信号が固定されない、すこし異なった状況である。

**Target Network**では、Q学習における教師信号、つまり新しい推定値( $R_{t+1}+\gamma \max_{a'}Q(s', a')$ )を一定ステップ固定することでこの問題の軽減を試みる。具体的には、通常のQ-Networkともう1つ同じ構造のTarget Networkを保持し、一定ステップことにQ-Networkにコピーすることでこれを実現する。

***記法整理**

今後使っていく記号を以下に記載しておく。
- $G_t$ : 利得、時刻t以降に獲得できる報酬の累積値
- $\gamma$ : 割引率
- $S_t$ : 時刻tにおける状態
- $A_t$ : 時刻tにおける行動
- $R_{t+1}$ : 時刻t+1における報酬
- $v_\pi(s)$ : 状態sにおける方策πの真の状態価値関数
- $V_\pi(s)$ : 状態sにおける方策πの推定状態価値関数
- $q_\pi(s, a)$ : 状態と行動のペア(s, a)における方策πの真の行動価値関数
- $Q_\pi(s, a)$ : 状態と行動のペア(s, a)における方策πの推定行動価値関数
- $A_\pi(s, a)$ : 状態と行動のペア(s, a)における方策πのアドバンテージ関数
- $y^{(i)}$ : 学習ステップiにおけるQ学習の教師信号
- $\delta^{(i)}$ : 学習ステップiにおけるTD誤差
- $\pi$ : 方策
- $\alpha$ : 学習のステップ幅
